In [1]:
using Revise
using Test

In [2]:
using LinearAlgebra
using ITensors
import CTMRG

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [3]:
ITensors.disable_warn_order()

14

In [4]:
L = 2
stag_list = CTMRG.make_CTMRG_ind_stags(L)
σtag_list = CTMRG.make_CTMRG_ind_σtags(L)

3×4 Matrix{Index}:
 (dim=2|id=530|"i=1,j=1,σ")  …  (dim=2|id=550|"i=1,j=4,σ")
 (dim=2|id=701|"i=2,j=1,σ")     (dim=2|id=348|"i=2,j=4,σ")
 (dim=2|id=342|"i=3,j=1,σ")     (dim=2|id=58|"i=3,j=4,σ")

In [5]:
#左下のみとりあえず関数を作成/1loop目は特殊処理
function setup_leftdown_corner_first(β::Float64, J::Float64,
    σindlist::Matrix{Index},sindlist::Matrix{Index})
    cindσ = combiner(σindlist[2, 1], σindlist[2, 2], tags="σ_all")
    cinds = combiner(sindlist[1, 2], sindlist[2, 2], tags="s_all")
    #2脚テンソルの定義
    C = CTMRG.C_corner(β, J, σindlist[1, 1], sindlist[1, 1])
    #3脚テンソルの定義
    P_edge = CTMRG.P_edge(β, J, σindlist[1, 1], σindlist[2, 1], sindlist[2, 1])
    P_corner = CTMRG.P_corner(β, J, σindlist[1, 2], sindlist[1, 1], sindlist[1, 2])
    #4脚テンソルの定義
    W =  CTMRG.ising_transfer_tensor_2D(β, J, σindlist[1, 2], σindlist[2, 2], sindlist[2, 1], sindlist[2, 2])
    #縮約
    _P_edge = P_edge * W
    C_expand = P_corner * C * _P_edge
    return C_expand,cindσ, cinds
end
#対角化した関数を作成
function diag_coner_tensor(C_expand::ITensor,cindσ::ITensor,
    cinds::ITensor)
    _C_expand = C_expand * cindσ * cinds
    C_array = array(_C_expand)
    evals, evecs = eigen(C_array)
    Λ, A, iA = CTMRG.make_diag_Tesnor(evals, evecs, inds(cindσ)[1], inds(cinds)[1])
    return Λ, A, iA
end


diag_coner_tensor (generic function with 1 method)

In [6]:
#左上を作成
function setup_leftup_corner_first(β::Float64, J::Float64,
    σindlist::Matrix{Index},sindlist::Matrix{Index})
    cindσ = combiner(σindlist[2, 1], σindlist[2, 2], tags="σ_all")
    cinds = combiner(sindlist[3, 2], sindlist[4, 2], tags="s_all")
    #2脚テンソルの定義
    C = CTMRG.C_corner(β, J, σindlist[3, 1], sindlist[4, 1])
    #3脚テンソルの定義
    P_edge = CTMRG.P_edge(β, J, σindlist[3, 1], σindlist[2, 1], sindlist[3, 1])
    P_corner = CTMRG.P_corner(β, J, σindlist[3, 2], sindlist[4, 1], sindlist[4, 2])
    #4脚テンソルの定義
    W =  CTMRG.ising_transfer_tensor_2D(β, J, σindlist[3, 2], σindlist[2, 2], sindlist[3, 1], sindlist[3, 2])
    #縮約
    _P_edge = P_edge * W
    C_expand = P_corner * C * _P_edge
    return C_expand, cindσ, cinds
end
#右上を作成
function setup_rightup_corner_first(β::Float64, J::Float64,
    σindlist::Matrix{Index},sindlist::Matrix{Index})
    cindσ = combiner(σindlist[2, 3], σindlist[2, 4], tags="σ_all")
    cinds = combiner(sindlist[3, 2], sindlist[4, 2], tags="s_all")
    #2脚テンソルの定義
    C = CTMRG.C_corner(β, J, σindlist[3, 4], sindlist[4, 3])
    #3脚テンソルの定義
    P_edge = CTMRG.P_edge(β, J, σindlist[3, 4], σindlist[2, 4], sindlist[3, 3])
    P_corner = CTMRG.P_corner(β, J, σindlist[3, 3], sindlist[4, 2], sindlist[4, 3])
    #4脚テンソルの定義
    W =  CTMRG.ising_transfer_tensor_2D(β, J, σindlist[2, 3], σindlist[3, 3], sindlist[3, 2], sindlist[3, 3])
    #縮約
    _P_edge = P_edge * W
    C_expand = P_corner * C * _P_edge
    return C_expand, cindσ, cinds
end
#右下を作成
function setup_rightdown_corner_first(β::Float64, J::Float64,
    σindlist::Matrix{Index},sindlist::Matrix{Index})
    cindσ = combiner(σindlist[2, 3], σindlist[2, 4], tags="σ_all")
    cinds = combiner(sindlist[1, 2], sindlist[2, 2], tags="s_all")
    #2脚テンソルの定義
    C = CTMRG.C_corner(β, J, σindlist[1, 4], sindlist[1, 3])
    #3脚テンソルの定義
    P_edge = CTMRG.P_edge(β, J, σindlist[2, 4], σindlist[1, 4], sindlist[2, 3])
    P_corner = CTMRG.P_corner(β, J, σindlist[1, 3], sindlist[1, 2], sindlist[1, 3])
    #4脚テンソルの定義
    W =  CTMRG.ising_transfer_tensor_2D(β, J, σindlist[1, 3], σindlist[2, 3], sindlist[2, 2], sindlist[2, 3])
    #縮約
    _P_edge = P_edge * W
    C_expand = P_corner * C * _P_edge
    return C_expand, cindσ, cinds
end


setup_rightdown_corner_first (generic function with 1 method)

In [7]:
C1,_,_ = setup_leftdown_corner_first(1.0, 1.0, σtag_list, stag_list)
C2,_,_ = setup_leftup_corner_first(1.0, 1.0, σtag_list, stag_list)
C3,_,_ = setup_rightup_corner_first(1.0, 1.0, σtag_list, stag_list)
C4,_,_ = setup_rightdown_corner_first(1.0, 1.0, σtag_list, stag_list)
@show only(C1*C2* C3*C4)
@show CTMRG.partition_function_2D(2, 1.0, 1.0)
;

only(C1 * C2 * C3 * C4) = 6.28697727090268e27
CTMRG.partition_function_2D(2, 1.0, 1.0) = 6.286977270902691e27


$$ 環境テンソルの作成

In [8]:
C1,σ_all1,s_all1 = setup_leftdown_corner_first(1.0, 1.0, σtag_list, stag_list)
C2,σ_all2,s_all2  = setup_leftup_corner_first(1.0, 1.0, σtag_list, stag_list)
@show typeof(inds(σ_all1)[2:end])
indσlist1 = collect(inds(σ_all1)[2:end])
indσlist2 = collect(inds(σ_all2)[2:end])
;

typeof((inds(σ_all1))[2:end]) = Tuple{Index{Int64}, Index{Int64}}


In [9]:
#L+1にするテンソルの作成
function σ_edge_tensor_left(β::Float64, J::Float64,
    σindlist_up::Vector{Index{Int64}},σindlist_down::Vector{Index{Int64}},
    indname::String)
    Tlist = []
    ind = Index(2, "$indname")
    sdumyind_list = [Index(2, "dummy,$i") for i in 1:length(σindlist_up)-1]
    if σindlist_down[1] == σindlist_up[1]
        _σindlist_down = [prime(σindlist_down[i]) for i in 1:length(σindlist_down)]
    else
        _σindlist_down = σindlist_down
    end

    for i in 1:length(σindlist_up)
        if i == 1
            T1 = CTMRG.P_edge(β, J, σindlist_up[1], _σindlist_down[1], sdumyind_list[1])
            push!(Tlist, T1)
        elseif i == length(σindlist_up)
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σindlist_up[i], _σindlist_down[i], sdumyind_list[i-1],ind)
            push!(Tlist, T1)
        else
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σindlist_up[i], _σindlist_down[i], sdumyind_list[i-1], sdummyind_list[i])
            push!(Tlist, T1)
        end
    end
    result = ITensor(true)
    for T in Tlist
        result *= T
    end
    return result, ind, _σindlist_down
end
function σ_edge_tensor_right(β::Float64, J::Float64,
    σindlist_up::Vector{Index{Int64}},σindlist_down::Vector{Index{Int64}},
    indname::String)
    Tlist = []
    ind = Index(2, "$indname")
    sdumyind_list = [Index(2, "dummy,$i") for i in 1:length(σindlist_up)-1]
    if σindlist_down[1] == σindlist_up[1]
        _σindlist_down = [prime(σindlist_down[i]) for i in 1:length(σindlist_down)]
    else
        _σindlist_down = σindlist_down
    end

    for i in 1:length(σindlist_up)
        if i == 1
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σindlist_up[1], _σindlist_down[1], sdumyind_list[i+1],ind)
            push!(Tlist, T1)
        elseif i == length(σindlist_up)
            T1 = CTMRG.P_edge(β, J, σindlist_up[i], _σindlist_down[i], sdumyind_list[i])
            push!(Tlist, T1)
        else
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σindlist_up[i], _σindlist_down[i], sdumyind_list[i-1], sdummyind_list[i])
            push!(Tlist, T1)
        end
    end
    result = ITensor(true)
    for T in Tlist
        result *= T
    end
    return result, ind, _σindlist_down
end


σ_edge_tensor_right (generic function with 1 method)

In [17]:
T, ind1, _σind_list = σ_edge_tensor_left(1.0, 1.0, indσlist1, indσlist2,"α")
T1 = T * σ_all1
_T1 = replaceinds(T1, Tuple(_σind_list)  => indσlist2) * σ_all2
;

In [20]:
function s_edge_tensor_down(β::Float64, J::Float64,
    sindlist_left::Vector{Index{Int64}},sindlist_right::Vector{Index{Int64}},
    indname::String)
    Tlist = []
    ind = Index(2, "$indname")
    σdumyind_list = [Index(2, "dummy,$i") for i in 1:length(sindlist_up)-1]
    if sindlist_down[1] == sindlist_up[1]
        _sindlist_right = [prime(sindlist_right[i]) for i in 1:length(sindlist_right)]
    else
        _sindlist_right = sindlist_right
    end

    for i in 1:length(sindlist_up)
        if i == 1
            T1 = CTMRG.P_corner(β, J, σdumyind_list[i], sindlist_left[1], _sindlist_down[1])
            push!(Tlist, T1)
        elseif i == length(sindlist_up)
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, ind, σdumyind_list[i-1], sindlist_left[i], _sindlist_down[i])
            push!(Tlist, T1)
        else
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σdumyind_list[i], σdumyind_list[i-1], sdumyind_list[i], sdumyind_list[i])
            push!(Tlist, T1)
        end
    end
    result = ITensor(true)
    for T in Tlist
        result *= T
    end
    return result, ind, _sindlist_down
end
function s_edge_tensor_up(β::Float64, J::Float64,
    sindlist_left::Vector{Index{Int64}},sindlist_right::Vector{Index{Int64}},
    indname::String)
    Tlist = []
    ind = Index(2, "$indname")
    σdumyind_list = [Index(2, "dummy,$i") for i in 1:length(sindlist_up)-1]
    if sindlist_down[1] == sindlist_up[1]
        _sindlist_right = [prime(sindlist_right[i]) for i in 1:length(sindlist_right)]
    else
        _sindlist_right = sindlist_right
    end

    for i in 1:length(sindlist_up)
        if i == 1
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, ind, σdumyind_list[i+1], sindlist_left[1], _sindlist_down[1]) ##
            push!(Tlist, T1)
        elseif i == length(sindlist_up)
            T1 = CTMRG.P_corner(β, J, σdumyind_list[i], sindlist_left[i], _sindlist_down[i])
            push!(Tlist, T1)
        else
            T1 = CTMRG.ising_transfer_tensor_2D(β, J, σdumyind_list[i], σdumyind_list[i+1], sdumyind_list[i], sdumyind_list[i])
            push!(Tlist, T1)
        end
    end
    result = ITensor(true)
    for T in Tlist
        result *= T
    end
    return result, ind, _sindlist_down
end

s_edge_tensor_up (generic function with 1 method)

In [22]:
#環境テンソルの作成
C1,σ_all1,s_all1= setup_leftdown_corner_first(1.0, 1.0, σtag_list, stag_list)
C2,σ_all2,s_all2= setup_leftup_corner_first(1.0, 1.0, σtag_list, stag_list)
C3,σ_all3,s_all3 = setup_rightup_corner_first(1.0, 1.0, σtag_list, stag_list)
C4,σ_all4,s_all4 = setup_rightdown_corner_first(1.0, 1.0, σtag_list, stag_list)
;

In [14]:
combiner(σindlist[2, 3], σindlist[2, 4], tags="σ_all")

UndefVarError: UndefVarError: `σindlist` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [15]:
σ_all1

ITensor ord=3 (dim=4|id=472|"σ_all") (dim=2|id=701|"i=2,j=1,σ") (dim=2|id=206|"i=2,j=2,σ")
NDTensors.Combiner

In [16]:
T1 = T * σ_all1

UndefVarError: UndefVarError: `T` not defined in `Main`
Suggestion: check for spelling errors or missing imports.